# READ ME
## INSTRUCTIONS
1. Make a copy of this file in and put your name next to it EX: writing_data_to_yourname (This is so we can do the same thing with multiple influx databases)
2. Create your database in influx, and a bucket for the data to go to (in this mine is called API Test)
3. Scroll down to the section at the bottom named "Change this to your database" and look for the comments #Replace with your own. Change token, org, bucket, and cloud_url (In influx you need to generate the API token for your database)
4. At the top toolbar go to runtime->restart session and run all
5. You will have to allow access to your drive so it can get the CSV file
6. Go down some towards the load data header where the location of the csv file is, this will have to be changed for your drive


The CSV file used in this code is a spreadsheet from one of the last groups measurements. Using the machine learning model, predictions were made for each appliance and added on. In the future, we would not have the columns named Actual/predicted_appliance name. We would only have measure the total power and then made predictions for each appliance.

## InfluxDB data handling integration
Instead of using csv as input file, we'll use the influxDB api to load in the data. Then we can use the machine learning model to disagregate it and write the new data back into influx. Reference on influxDB API here: https://www.influxdata.com/blog/time-series-forecasting-with-tensorflow-influxdb/

In [1]:
! pip install influxdb-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.0/744.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 17.9 MB/s eta 0:00:00


Library dependencies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

load data

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# List of CSV files to use for training
#
# READ ME
# on the left side of your screen click on the folder icon. Run the previous code cell that says mount google drive or click the folder with the drive icon on the left side.
# Navigate to our senior design folder then follow the same path to find the prediction_vs_actual.csv file. Right click on it to copy the path to the file then replace the one below
csv_files = ['/content/drive/MyDrive/50_ResidentialPowerDisaggregation_SD_Fall23/1.2 Software/Colab Notebooks/ML Models/Andrew/prediction_vs_actual.csv']  # Add more file names as needed

# Load and concatenate data from multiple CSV files
data_list = []
for csv_file in csv_files:
    data = pd.read_csv(csv_file)
    data_list.append(data)

# Concatenate data from all CSV files
data = pd.concat(data_list, ignore_index=True)

This prints the orignal data from the CSV file

In [5]:
data

,timestamp,Total,Actual_Washer,Actual_BlowerGH,Actual_Lights,Actual_BlowerBed,Actual_CompGH,Actual_CompBed,Actual_Dryer,Actual_Recs1,...,Predicted_BlowerGH,Predicted_Lights,Predicted_BlowerBed,Predicted_CompGH,Predicted_CompBed,Predicted_Dryer,Predicted_Recs1,Predicted_Recs2,Predicted_WaterHeater,Timestamp
0,NaN,52.6,0.6,4.9,25.9,2.0,0.0,5.3,0.0,8.3,...,5.458139,50.377780,-8.722680,4.118182,129.59215,-7.054536,23.361870,-20.186514,8.394237,2024-01-22 03:02:00
1,NaN,968.6,0.7,6.2,25.4,227.8,21.1,672.7,0.0,8.8,...,12.681525,49.016400,37.552692,44.333454,161.14293,1.098680,24.044157,-9.713423,15.486119,2024-01-22 03:03:00
2,NaN,910.3,0.7,6.1,25.4,227.5,21.0,615.6,0.0,8.1,...,26.024382,48.511833,253.663470,102.330124,316.01540,16.988558,24.677510,6.385436,26.801900,2024-01-22 03:04:00
3,NaN,867.3,0.6,6.1,25.5,228.5,21.0,571.6,0.0,8.1,...,27.296140,53.936330,355.601260,112.400040,439.14758,14.003014,26.278265,0.368196,2.480625,2024-01-22 03:05:00
4,NaN,911.4,0.7,6.1,25.5,229.9,20.9,614.3,0.0,8.1,...,24.843350,61.003914,328.448600,130.901280,616.89700,-3.106190,27.205340,-15.704743,-45.088432,2024-01-22 03:06:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,NaN,1924.6,6.4,8.9,41.7,225.5,25.2,629.4,970.1,10.1,...,120.094680,32.020508,902.356100,600.841700,685.75290,37.130672,4.813627,26.974987,27.149942,2024-01-22 22:11:00
1150,NaN,944.8,6.0,6.3,43.4,226.8,21.1,628.1,0.0,7.3,...,51.698124,44.912987,554.754200,165.974460,632.74900,32.808086,11.331118,9.906139,27.406240,2024-01-22 22:12:00
1151,NaN,949.1,6.0,6.4,43.5,227.9,21.2,631.0,0.0,7.3,...,41.504795,46.085804,554.549600,125.079475,677.70750,27.612118,10.222755,4.091804,18.958097,2024-01-22 22:13:00
1152,NaN,1981.5,6.4,9.2,41.9,225.9,25.6,628.0,1026.7,10.3,...,117.369200,36.114090,867.376000,584.114140,687.21250,27.567614,7.031342,-0.502177,1.388543,2024-01-22 22:14:00


Data has to be changed so it is within the 30 day retention period, this was already done for the csv file

In [6]:
data = data.drop(columns=['timestamp'])

In [7]:
data

,Total,Actual_Washer,Actual_BlowerGH,Actual_Lights,Actual_BlowerBed,Actual_CompGH,Actual_CompBed,Actual_Dryer,Actual_Recs1,Actual_Recs2,...,Predicted_BlowerGH,Predicted_Lights,Predicted_BlowerBed,Predicted_CompGH,Predicted_CompBed,Predicted_Dryer,Predicted_Recs1,Predicted_Recs2,Predicted_WaterHeater,Timestamp
0,52.6,0.6,4.9,25.9,2.0,0.0,5.3,0.0,8.3,5.6,...,5.458139,50.377780,-8.722680,4.118182,129.59215,-7.054536,23.361870,-20.186514,8.394237,2024-01-22 03:02:00
1,968.6,0.7,6.2,25.4,227.8,21.1,672.7,0.0,8.8,5.9,...,12.681525,49.016400,37.552692,44.333454,161.14293,1.098680,24.044157,-9.713423,15.486119,2024-01-22 03:03:00
2,910.3,0.7,6.1,25.4,227.5,21.0,615.6,0.0,8.1,5.9,...,26.024382,48.511833,253.663470,102.330124,316.01540,16.988558,24.677510,6.385436,26.801900,2024-01-22 03:04:00
3,867.3,0.6,6.1,25.5,228.5,21.0,571.6,0.0,8.1,5.9,...,27.296140,53.936330,355.601260,112.400040,439.14758,14.003014,26.278265,0.368196,2.480625,2024-01-22 03:05:00
4,911.4,0.7,6.1,25.5,229.9,20.9,614.3,0.0,8.1,5.9,...,24.843350,61.003914,328.448600,130.901280,616.89700,-3.106190,27.205340,-15.704743,-45.088432,2024-01-22 03:06:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,1924.6,6.4,8.9,41.7,225.5,25.2,629.4,970.1,10.1,7.3,...,120.094680,32.020508,902.356100,600.841700,685.75290,37.130672,4.813627,26.974987,27.149942,2024-01-22 22:11:00
1150,944.8,6.0,6.3,43.4,226.8,21.1,628.1,0.0,7.3,5.8,...,51.698124,44.912987,554.754200,165.974460,632.74900,32.808086,11.331118,9.906139,27.406240,2024-01-22 22:12:00
1151,949.1,6.0,6.4,43.5,227.9,21.2,631.0,0.0,7.3,5.8,...,41.504795,46.085804,554.549600,125.079475,677.70750,27.612118,10.222755,4.091804,18.958097,2024-01-22 22:13:00
1152,1981.5,6.4,9.2,41.9,225.9,25.6,628.0,1026.7,10.3,7.5,...,117.369200,36.114090,867.376000,584.114140,687.21250,27.567614,7.031342,-0.502177,1.388543,2024-01-22 22:14:00


# Change this to your database

In [8]:
# Provide connection details

#Replace with your own
token = "qSgiUTIV1MSmpwOENTRpKvUGmeD1czVVBhxw7ABPZNJHEykiwRoRchQDl4drwQyzDo3aKYi4JvHyS6Y-o9TeFQ=="
#Replace with your own
org = "NCSU Senior Design Project 50"
#Replace with your own
bucket = "Data Test"

#Replace with your own
# InfluxDB Cloud URL
cloud_url = "https://us-east-1-1.aws.cloud2.influxdata.com"

# Establish InfluxDB connection
client = InfluxDBClient(url=cloud_url, token=token, org=org)

# Check if 'timestamp' and 'value' columns are present
if 'Timestamp' not in data.columns:
    raise ValueError("Column 'Timestamp' is required in the DataFrame.")

# Convert DataFrame to InfluxDB Points
points = data.apply(lambda row: Point("NILM")
                                     .field("Total", row["Total"])
                                     .field("Actual_Washer", row["Actual_Washer"])
                                     .field("Predicted_Washer", row["Predicted_Washer"])
                                     .field("Actual_BlowerGH", row["Actual_BlowerGH"])
                                     .field("Predicted_BlowerGH", row["Predicted_BlowerGH"])
                                     .field("Actual_Lights", row["Actual_Lights"])
                                     .field("Predicted_Lights", row["Predicted_Lights"])
                                     .field("Actual_BlowerBed", row["Actual_BlowerBed"])
                                     .field("Predicted_BlowerBed", row["Predicted_BlowerBed"])
                                     .field("Actual_CompGH", row["Actual_CompGH"])
                                     .field("Predicted_CompGH", row["Predicted_CompGH"])
                                     .field("Actual_CompBed", row["Actual_CompBed"])
                                     .field("Predicted_CompBed", row["Predicted_CompBed"])
                                     .field("Actual_Dryer", row["Actual_Dryer"])
                                     .field("Predicted_Dryer", row["Predicted_Dryer"])
                                     .field("Actual_Recs1", row["Actual_Recs1"])
                                     .field("Predicted_Recs1", row["Predicted_Recs1"])
                                     .field("Actual_Recs2", row["Actual_Recs2"])
                                     .field("Predicted_Recs2", row["Predicted_Recs2"])
                                     .field("Actual_WaterHeater", row["Actual_WaterHeater"])
                                     .field("Predicted_WaterHeater", row["Predicted_WaterHeater"])
                                     .time(row["Timestamp"]), axis=1)

# Create a write API instance
write_api = client.write_api(write_options=SYNCHRONOUS)

# Write Points to InfluxDB
write_api.write(bucket=bucket, record=points)